# Transforming files from CVS to TFRecords
Here we are transforming our test and training data to TFRecords, so that we can use it for model training.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [9]:
import os, gc
import contextlib2
from object_detection.dataset_tools import tf_record_creation_util

from tqdm import tqdm
tqdm.pandas()

In [7]:
#!pip install object_detection

     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     - -------------------------------------- 0.1/1.5 MB 1.1 MB/s eta 0:00:02
     --- ------------------------------------ 0.1/1.5 MB 1.6 MB/s eta 0:00:01
     ------ --------------------------------- 0.2/1.5 MB 1.7 MB/s eta 0:00:01
     --------- ------------------------------ 0.4/1.5 MB 2.2 MB/s eta 0:00:01
     ------------- -------------------------- 0.5/1.5 MB 2.3 MB/s eta 0:00:01
     ---------------------- ----------------- 0.8/1.5 MB 3.0 MB/s eta 0:00:01
     ------------------------------------- -- 1.4/1.5 MB 4.4 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 4.1 MB/s eta 0:00:00


In [3]:
#!pip install contextlib2

In [10]:
df_train = pd.read_csv('train_data_QUICK_START.csv')
print(df_train.shape)
df_train.head()

(335616, 416)


,sequence_id,sequence,experiment_type,dataset_name,reactivity_0001,reactivity_0002,reactivity_0003,reactivity_0004,reactivity_0005,reactivity_0006,...,reactivity_error_0197,reactivity_error_0198,reactivity_error_0199,reactivity_error_0200,reactivity_error_0201,reactivity_error_0202,reactivity_error_0203,reactivity_error_0204,reactivity_error_0205,reactivity_error_0206
0,0000d87cab97,GGGAACGACUCGAGUAGAGUCGAAAAAGAUCGCCACGCACUUACGA...,2A3_MaP,DasLabBigLib_OneMil_RFAM_windows_100mers_2A3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000d87cab97,GGGAACGACUCGAGUAGAGUCGAAAAAGAUCGCCACGCACUUACGA...,DMS_MaP,DasLabBigLib_OneMil_RFAM_windows_100mers_DMS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0001ca9d21b0,GGGAACGACUCGAGUAGAGUCGAAAAGGUGGCCGGCAGAAUCGCGA...,2A3_MaP,DasLabBigLib_OneMil_OpenKnot_Round_2_train_2A3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0001ca9d21b0,GGGAACGACUCGAGUAGAGUCGAAAAGGUGGCCGGCAGAAUCGCGA...,DMS_MaP,DasLabBigLib_OneMil_OpenKnot_Round_2_train_DMS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00021f968267,GGGAACGACUCGAGUAGAGUCGAAAACAUUGUUAAUGCCUAUAUUA...,2A3_MaP,DasLabBigLib_OneMil_Replicates_from_previous_l...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
train = df_train.sequence.to_numpy()
encoding_dict = {'A':1, 'C': 2, 'G': 3, 'U': 4}
encoding_dict

{'A': 1, 'C': 2, 'G': 3, 'U': 4}

In [12]:
# TFrecordsiks kuidagi aga idk kuidas
# Näide (?) https://www.kaggle.com/code/irohith/aslfr-preprocess-dataset/notebook
# https://www.kaggle.com/code/konstantinboyko/convert-original-csv-file-to-tfrecord

In [13]:
lo_react_cols = df_train.filter(like='reactivity_0').columns
lo_error_cols = df_train.filter(like='reactivity_error_0').columns

lst_react_cols = lo_react_cols.to_list()
lst_error_cols = lo_error_cols.to_list()

In [15]:
print(lst_react_cols)
print(lst_error_cols)

['reactivity_0001', 'reactivity_0002', 'reactivity_0003', 'reactivity_0004', 'reactivity_0005', 'reactivity_0006', 'reactivity_0007', 'reactivity_0008', 'reactivity_0009', 'reactivity_0010', 'reactivity_0011', 'reactivity_0012', 'reactivity_0013', 'reactivity_0014', 'reactivity_0015', 'reactivity_0016', 'reactivity_0017', 'reactivity_0018', 'reactivity_0019', 'reactivity_0020', 'reactivity_0021', 'reactivity_0022', 'reactivity_0023', 'reactivity_0024', 'reactivity_0025', 'reactivity_0026', 'reactivity_0027', 'reactivity_0028', 'reactivity_0029', 'reactivity_0030', 'reactivity_0031', 'reactivity_0032', 'reactivity_0033', 'reactivity_0034', 'reactivity_0035', 'reactivity_0036', 'reactivity_0037', 'reactivity_0038', 'reactivity_0039', 'reactivity_0040', 'reactivity_0041', 'reactivity_0042', 'reactivity_0043', 'reactivity_0044', 'reactivity_0045', 'reactivity_0046', 'reactivity_0047', 'reactivity_0048', 'reactivity_0049', 'reactivity_0050', 'reactivity_0051', 'reactivity_0052', 'reactivity

In [16]:
del lo_error_cols, lo_react_cols

In [17]:
df_train['reactivity'] = df_train[lst_react_cols].values.tolist()
df_train['error'] = df_train[lst_error_cols].values.tolist()

In [18]:
df_train.drop(columns=lst_react_cols,inplace=True)
df_train.drop(columns=lst_error_cols,inplace=True)

In [19]:
all_cols = df_train.columns

idx_seq_id          = all_cols.get_loc('sequence_id')
idx_sequence        = all_cols.get_loc('sequence')
idx_dataset_name    = all_cols.get_loc('dataset_name')
idx_reactivity      = all_cols.get_loc('reactivity')
idx_error           = all_cols.get_loc('error')

In [20]:
df_2A3 = df_train.loc[df_train.experiment_type=='2A3_MaP'].reset_index(drop=True)
df_DMS = df_train.loc[df_train.experiment_type=='DMS_MaP'].reset_index(drop=True)

del df_train
gc.collect()

0

In [21]:
def int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def int64_list_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def bytes_list_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

def float_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def float_list_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def string_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))

def string_list_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=value.encode()))

In [30]:
def serialize_row(row_2A3, row_DMS):
  """
  Creates a tf.train.Example message ready to be written to a file.
  """
  
  assert row_2A3[idx_seq_id] == row_DMS[idx_seq_id] and row_2A3[idx_sequence] == row_DMS[idx_sequence]
    
  # Create a dictionary mapping the feature name to the tf.train.Example-compatible data type.
  feature = {
      "id": string_feature(row_2A3[idx_seq_id]),
      "seq": float_list_feature([encoding_dict[char] for char in row_2A3[idx_sequence]]), 

      "dataset_name_2A3": string_feature(row_2A3[idx_dataset_name]),
      "reactivity_2A3": float_list_feature(row_2A3[idx_reactivity]),
      "error_2A3": float_list_feature(row_2A3[idx_error]),

      "dataset_name_DMS": string_feature(row_DMS[idx_dataset_name]),
      "reactivity_DMS": float_list_feature(row_DMS[idx_reactivity]),
      "error_DMS": float_list_feature(row_DMS[idx_error])
  }

  # Create a Features message using tf.train.Example.
  features=tf.train.Features(feature=feature)
  proto_example = tf.train.Example(features=features)
  return proto_example

In [23]:
#Utilities for creating TFRecords of TF examples for the Open Images dataset.
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

def open_sharded_output_tfrecords(exit_stack, base_path, num_shards):
  """Opens all TFRecord shards for writing and adds them to an exit stack.

  Args:
    exit_stack: A context2.ExitStack used to automatically closed the TFRecords
      opened in this function.
    base_path: The base path for all shards
    num_shards: The number of shards

  Returns:
    The list of opened TFRecords. Position k in the list corresponds to shard k.
  """
  tf_record_output_filenames = ['{}/{:03d}.tfrecord'.format(base_path, idx) for idx in range(num_shards)]

  # options = tf.io.TFRecordOptions(compression_type="GZIP")
  tfrecords = [
      exit_stack.enter_context(tf.io.TFRecordWriter(file_name, options="GZIP"))
          for file_name in tf_record_output_filenames
  ]

  return tfrecords

In [31]:
num_shards=164
output_filebase='tfrfile'

with contextlib2.ExitStack() as tf_record_close_stack:
    output_tfrecords = open_sharded_output_tfrecords(tf_record_close_stack, output_filebase, num_shards)
    for index, (row_2A3, row_DMS) in enumerate(zip(df_2A3.itertuples(index=False), df_DMS.itertuples(index=False))):
        tf_example = serialize_row(row_2A3, row_DMS)
        shard_index = index % num_shards
        output_tfrecords[shard_index].write(tf_example.SerializeToString())



In [37]:
def parse_tfrecord(tfrecord_file):
    raw_dataset = tf.data.TFRecordDataset(tfrecord_file)

    for raw_record in raw_dataset.take(5):  # Read the first 5 records
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        print(example)  # Print the parsed example (protobuf message)

# Call the function to parse and inspect the TFRecord file
parse_tfrecord('tfrfil/001.tfrecord')

NotFoundError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} NewRandomAccessFile failed to Create/Open: tfrfil/001.tfrecord : The system cannot find the path specified.
; No such process [Op:IteratorGetNext] name: 